All of the functions below require numpy:

In [1]:
#importing library 

import numpy as np

In [2]:
#defing the arrays for rate, price and hardware requirements

rates = np.array([5, 5, 1.8, 3, 2.6, 1, 0.8, 0.4, 0.3])
price = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])
requirements = np.array([[16,8,1],[32,16,1],[64,32,1],[8,32,1],[16,64,1],[32,128,1],[16,16,2],[32,32,6],[64,64,8]])

In [3]:
#defing total per day CPU, memory, GPU 

C = 512
M = 1024
G = 64
T = 5

# Computing the forecasted demand for each fare type:
forecast = T * rates
forecast
ninstances = 9

In [4]:
# Part 2
from gurobipy import *

# Create the model
m = Model()

# Create variables.
x = m.addVars(ninstances, lb = 0, ub = forecast)

# CPU constraint
cpu_constr = m.addConstr( sum(x[i]*requirements[i,0] for i in range(ninstances)) <= C )
# Memory constraint
memory_constr = m.addConstr( sum(x[i]*requirements[i,1] for i in range(ninstances)) <= M )
# GPU constraint
gpu_constr = m.addConstr( sum(x[i]*requirements[i,2] for i in range(ninstances)) <= G )

# Set the objective.
m.setObjective(sum( price[i] * x[i] for i in range(ninstances)), GRB.MAXIMIZE )

# Update + solve:
m.update()
m.optimize()

# Get the objective value
LP_obj = m.objval

# Get the allocation
allocation = np.array([x[i].x for i in range(ninstances)])

# Display the results:
print("Allocation =",allocation)
print("Forecast =",forecast)
print("Objective =",LP_obj)

Academic license - for non-commercial use only
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [2e+00, 3e+01]
  RHS range        [6e+01, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 3 rows, 8 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4400000e+03   7.200000e+01   0.000000e+00      0s
       3    1.0394286e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  1.039428571e+03
Allocation = [6.28571429 0.         0.         3.42857143 0.         5.
 4.         2.         1.5       ]
Forecast = [25.  25.   9.  15.  13.   5.   4.   2.   1.5]
Objective = 1039.4285714285716


In [5]:
#Part 2 (c)

# cpu_constr.pi
if(gpu_constr.pi>0):
    print("Increasing GPU makes sense")
else:
    print("Increasing GPU does not makes sense")

#Part 2d
# Assessing whether increaed server capacity makes sense
#cost of renting for 5 days
cost = 5
additional_rev = 32*cpu_constr.pi + 16*memory_constr.pi
if(additional_rev>cost):
    print("Cirrus should rent the server")
else:
    print("Cirrus should not rent the server")

Increasing GPU does not makes sense
Cirrus should rent the server


## Function to generate arrival sequences (Q1, Part 3)

In [6]:
# Preconditions:
# nSimulations = integer specifying number of simulations to run
# rates = array containing arrival rate (# / day) for each of the instance
# types (should be an array with 9 elements)
# T = length of horizon in days.

def generateArrivalSequences( nSimulations, rates, T ):
    total_rate = sum(rates)
    nTypes = len(rates)

    arrival_sequences_times = []
    arrival_sequences_types = [];

    for s in range(nSimulations):
        single_arrival_sequence_time = [];
        single_arrival_sequence_type = [];
        t = 0;
        while (t < T):
            single_time = np.random.exponential(1.0/total_rate)
            single_type = np.random.choice(nTypes, p= rates/total_rate )

            t += single_time;

            if (t < T):
                single_arrival_sequence_time.append(t)
                single_arrival_sequence_type.append(single_type)
            else:
                break

        arrival_sequences_times.append(np.array(single_arrival_sequence_time))
        arrival_sequences_types.append(np.array(single_arrival_sequence_type))
    return arrival_sequences_times, arrival_sequences_types



In [7]:


np.random.seed(10)
nSimulations_test = 100
rates_test = rates
T_test = 5
arrival_sequences_times, arrival_sequences_types = generateArrivalSequences(nSimulations_test, rates_test, T_test)
arrival_sequences_times[0][0:3]

array([0.07414243, 0.1246028 , 0.15928449])

In [ ]:
# Part 3(a)

In [8]:

avg_c1 = 0
for i in range(100):
    avg_c1 = avg_c1+sum(arrival_sequences_types[i] ==0)
avg_c1 = avg_c1/100
print("Average arrivals for C1 = ",avg_c1)

Average arrivals for C1 =  26.63


In [ ]:
#Part 3b

In [9]:

import pandas as pd
avg_instances = {}
for i in range(len(rates)):
    avg_instances[i] = 0
for i in range(nSimulations_test):
    for j in range(len(rates)):
        avg_instances[j] =avg_instances[j]+ sum(arrival_sequences_types[i]==j)/nSimulations_test
# avg_instances = avg_instances/len(rates)
avg_instances

{0: 26.629999999999995,
 1: 24.380000000000013,
 2: 9.010000000000005,
 3: 14.880000000000004,
 4: 13.320000000000002,
 5: 4.929999999999998,
 6: 4.199999999999998,
 7: 2.2699999999999996,
 8: 1.3800000000000008}

## Skeleton code for function to run myopic policy (Q1, Part 3)

In [10]:
# Preconditions for code below:
nSimulations = 100
nResources = 3
# B = numpy array of initial capacities of the resources
# arrival_sequences_times = array where each entry is arrival time sequence for that
# simulation
# arrival_sequences_types = array where each entry is sequence of request types for
# that simulation

# Note: code will not run; parts with ... need to be filled in.

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
#     b = B.copy();
    single_revenue = 0.0; # will contain the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s]);
    C_remaining = C
    M_remaining = M
    G_remaining = G
    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (i)
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is sufficient capacity for the request
        if ( (C_remaining - requirements[i,0])>=0 and (M_remaining - requirements[i,1])>=0 and (G_remaining - requirements[i,2])>=0):
            single_revenue = single_revenue + price[i]
            C_remaining = C_remaining - requirements[i,0]
            M_remaining = M_remaining - requirements[i,1]
            G_remaining = G_remaining - requirements[i,2]

    # Save the results of this simulation here ...
    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[0,s] = C_remaining
    results_myopic_remaining_capacity[1,s] = M_remaining
    results_myopic_remaining_capacity[2,s] = G_remaining

# Find the average revenue
print("Average revenue",np.mean(results_myopic_revenue))
# Find the average remaining quantity of each resource
print("Average remaining capacity of cpu", np.mean(results_myopic_remaining_capacity[0,:]))
print("Average remaining capacity of memory", np.mean(results_myopic_remaining_capacity[1,:]))
print("Average remaining capacity of gpu", np.mean(results_myopic_remaining_capacity[2,:]))

Average revenue 528.28
Average remaining capacity of cpu 0.24
Average remaining capacity of memory 371.52
Average remaining capacity of gpu 37.42


## Skeleton code for function to run bid-price control policy (Q1, Part 4)

In [11]:
# Preconditions for code below:
# nSimulations = number of simulations to run
nResources = 3
nInstances = len(rates)
# B = numpy array of initial capacities of the resources
# arrival_sequences_times = array where each entry is arrival time sequence for that
# simulation
# arrival_sequences_types = array where each entry is sequence of request types for
# that simulation

# Your LP formulation should be defined here
# (or somewhere before the definition of bpc()...)
# ...


# As we did in-class, define a function bpc() to re-solve the LO each time:
def bpc(b, t):
    for r in range(nResources):
        # Set the RHS of the resource constraint to b[r] here
        cpu_constr.rhs = b[0]
        memory_constr.rhs = b[1]
        gpu_constr.rhs = b[2]

    for i in range(nInstances):
        # Set the RHS of the forecast constraint for each instance
        # type to the expected number of requests over the duration
        # of the remaining horizon (T - t).
        x[i].ub = (T - t)*rates[i]

    # Re-solve the model:
    m.update()
    m.optimize()

    # Obtain the dual values/shadow prices
    dual_val = [cpu_constr.pi,memory_constr.pi,gpu_constr.pi]

    # Return the dual values:
    return dual_val



results_revenue = np.zeros(nSimulations)
results_remaining_capacity = np.zeros((nResources, nSimulations))
B = [C,M,G]
for s in range(nSimulations):
    b = B.copy() #Initialize the current capacity
    single_revenue = 0.0 #Initialize the revenue garnered in this simulation
    nArrivals = len(arrival_sequences_times[s])
    for j in range(nArrivals):
        # Take the next arrival time and type from the sequence
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is enough capacity
        if ( (b[0] - requirements[i,0])>=0 and (b[1] - requirements[i,1])>=0 and (b[2] - requirements[i,2])>=0 ):
            # Re-solve the LO and obtain the dual values
            dual_val = bpc(b, arrival_time)

            # Check if the revenue is at least the sum of the bid prices:
            sum_of_bid =  dual_val[0]*requirements[i,0] + dual_val[1]*requirements[i,1] + dual_val[2]*requirements[i,2]
            if (price[i] >= sum_of_bid):
                # If there is sufficient capacity, accrue the revenue
                # and remove the capacity.
                single_revenue = single_revenue + price[i]
                b[0] = b[0] - requirements[i,0]
                b[1] = b[1] - requirements[i,1]
                b[2] = b[2] - requirements[i,2]

    # Save the results of this simulation here:
    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[0,s] = b[0]
    results_myopic_remaining_capacity[1,s] = b[1]
    results_myopic_remaining_capacity[2,s] = b[2]


# Find the average revenue
print("Average revenue",np.mean(results_myopic_revenue))
# Find the average remaining quantity of each resource
print("Average remaining capacity of cpu", np.mean(results_myopic_remaining_capacity[0,:]))
print("Average remaining capacity of memory", np.mean(results_myopic_remaining_capacity[1,:]))
print("Average remaining capacity of gpu", np.mean(results_myopic_remaining_capacity[2,:]))

Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [6e+01, 1e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0348614e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.034861397e+03
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [6e+01, 1e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0247530e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.024753039e+03
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds ran

Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+01, 7e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2241449e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.224144900e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+01, 6e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.9795929e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  6.979592876e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+01, 6e+02]


Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9417927e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.941792666e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [9e-01, 2e+01]
  RHS range        [4e+01, 2e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9291021e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.929102053e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [9e-01, 2e+01]
  RHS range        [4e+01, 2e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9234920e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01


Solved in 0 iterations and 0.01 seconds
Optimal objective  2.094931002e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [7e-01, 1e+01]
  RHS range        [3e+01, 1e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0942414e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.094241390e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [7e-01, 1e+01]
  RHS range        [3e+01, 1e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0811557e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.081155714e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Ma

Optimal objective  1.103842014e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [5e-01, 8e+00]
  RHS range        [3e+01, 7e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1036541e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.103654081e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [5e-01, 8e+00]
  RHS range        [3e+01, 7e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0954171e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.095417144e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective

...

  RHS range        [5e+01, 6e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5514708e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective  6.551470809e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+01, 6e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5096922e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  6.509692163e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+01, 6e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4972731e+02   0.000000e+00   0.000000e+00      0


Solved in 0 iterations and 0.02 seconds
Optimal objective  7.556750121e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+01, 7e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2866010e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.286601049e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+01, 7e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2454297e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.245429717e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Ma


Solved in 1 iterations and 0.01 seconds
Optimal objective  3.791217876e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [9e-01, 2e+01]
  RHS range        [3e+01, 3e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5710188e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  3.571018812e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [9e-01, 2e+01]
  RHS range        [3e+01, 2e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1280466e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  3.128046558e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Ma

  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [6e-01, 1e+01]
  RHS range        [2e+01, 7e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1489530e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  1.148952997e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [6e-01, 1e+01]
  RHS range        [2e+01, 7e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1460038e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.146003792e+02
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [6e-01, 1e+01]
  RHS range        [2e+01, 7e+01]
Iteration    Objective  

       0    6.7685994e+01   4.132027e-01   0.000000e+00      0s
       1    6.4380372e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  6.438037217e+01
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [2e-01, 3e+00]
  RHS range        [2e+01, 6e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7995078e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  5.799507783e+01
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e-01, 2e+00]
  RHS range        [2e+01, 6e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.4309200e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01

In [14]:
print("Average revenue",np.mean(results_myopic_revenue))
# Find the average remaining quantity of each resource
print("Average remaining capacity of cpu", np.mean(results_myopic_remaining_capacity[0,:]))
print("Average remaining capacity of memory", np.mean(results_myopic_remaining_capacity[1,:]))
print("Average remaining capacity of gpu", np.mean(results_myopic_remaining_capacity[2,:]))

Average revenue 925.59
Average remaining capacity of cpu 27.2
Average remaining capacity of memory 4.88
Average remaining capacity of gpu 20.62
